In [135]:
import yaml
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from matplotlib.pyplot import get_cmap

cmap=get_cmap("tab10")

In [136]:
with open("../../../results/20240131-test/diarization.yaml", "r") as f:
    diarization = yaml.safe_load(f)

speakers = []
for item in diarization:
    speakers.append(item["speaker"])

In [137]:
with open("../../../results/20240131-test/result.yaml", "r") as f:
    conversation = yaml.safe_load(f)
# conversation

In [168]:
line_conv_x, line_conv_y, line_x, line_y = [], [], [], []
num_speakers = len(set(speakers))
for _ in range(num_speakers):
    line_x.append([])
    line_y.append([])

In [169]:
_conv = []
for segment in conversation["segments"]:
    _conv.append(["Dialogue", segment["start"], segment["end"]])
    line_conv_x.extend([segment["start"], segment["end"], None])
    line_conv_y.extend(["Dialogue", "Dialogue", None])
df_conv = pd.DataFrame(_conv, columns=["Speaker", "Start", "End"])
# df_conv

In [171]:
_diarization = []
for _ in range(num_speakers):
    _diarization.append([])
    
for item in diarization:
    start = np.round(item["start"], 1)
    end   = np.round(item["end"], 1)
    label = item["speaker"]
    idx   =int(item["speaker"].split("_")[-1])
    _diarization[idx].append([label, start, end])
    line_x[idx].extend([start, end, None])
    line_y[idx].extend([label, label, None])
df_diars = [pd.DataFrame(_diarization[idx], columns=df_conv.columns) for idx in range(num_speakers)]
# df_diar

In [200]:
data = [go.Scatter(
            x=line_x[idx],
            y=line_y[idx],
            mode="lines",
            name=f"SP-{idx}",
            marker=dict(
                color=cmap(idx/10)
            )
        ) for idx in range(num_speakers)]
data.extend([
    go.Scatter(
        x=df_diars[idx]["Start"],
        y=df_diars[idx]["Speaker"],
        mode="markers",
        showlegend=False,
        marker=dict(
            color="black",#cmap(idx/10),
            size=5,
            symbol="triangle-right"
        )
    ) for idx in range(num_speakers)])

data.extend([
    go.Scatter(
        x=df_diars[idx]["End"],
        y=df_diars[idx]["Speaker"],
        mode="markers",
        showlegend=False,
        marker=dict(
            color="black",#cmap(idx/10),
            size=5,
            symbol="triangle-left"
        )
    ) for idx in range(num_speakers)])
data.extend([
    go.Scatter(
        x=line_conv_x,
        y=line_conv_y,
        mode="lines",
        name="dialogue",
        marker=dict(
            color="grey"
        )
    ),
    go.Scatter(
        x=df_conv["Start"],
        y=df_conv["Speaker"],
        mode="markers",
        showlegend=False,
        marker=dict(
            color="grey",
            size=5,
            symbol="triangle-right"
        )
    ),
    go.Scatter(
        x=df_conv["End"],
        y=df_conv["Speaker"],
        mode="markers",
        showlegend=False,
        marker=dict(
            color="grey",
            size=5,
            symbol="triangle-left"
        )
    )
])

In [201]:
fig = go.Figure(data=data)

In [203]:
fig.update_layout(
    title="Dialogue",
    height=300,
    legend_itemclick=False
)

fig.show()